# Interactive AGI simulation

- AI Velocity Gap Simulator
- This code creates an interactive simulation to visualize the "Velocity Gap" between harmful AI technologies and


In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, widgets

def plot_velocity_gap(harm_growth, benefit_ceiling, inst_speed, policy_lag):
    time = np.linspace(0, 25, 250)

    # Models
    y_harm = 0.5 * np.exp(harm_growth * time)
    midpoint = 10 + policy_lag
    y_benefit = benefit_ceiling / (1 + np.exp(-inst_speed * (time - midpoint)))

    # Calculate Risk Score (Area between curves)
    risk_score = np.trapz(np.maximum(0, y_harm - y_benefit), time)

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(time, y_harm, color='red', lw=2, label='Harmful AI Potential')
    plt.plot(time, y_benefit, color='blue', lw=2, label='Realized AI Benefits')

    # Fill the gap
    plt.fill_between(time, y_benefit, y_harm, where=(y_harm > y_benefit),
                     color='red', alpha=0.1, label='The Velocity Gap')

    # Formatting
    plt.title(f"AI Velocity Gap | Cumulative Risk: {risk_score:.2f}", fontsize=14)
    plt.xlabel("Years from AGI Emergence")
    plt.ylabel("Impact Magnitude")
    plt.ylim(0, min(max(y_harm)*1.1, 300))
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()

    plt.show()

# Interactive Sliders
interact(
    plot_velocity_gap,
    harm_growth = widgets.FloatSlider(value=0.25, min=0.1, max=0.4, step=0.01),
    benefit_ceiling = widgets.IntSlider(value=50, min=10, max=100, step=5),
    inst_speed = widgets.FloatSlider(value=0.4, min=0.1, max=1.0, step=0.05),
    policy_lag = widgets.IntSlider(value=0, min=-5, max=10, step=1)
);

interactive(children=(FloatSlider(value=0.25, description='harm_growth', max=0.4, min=0.1, step=0.01), IntSlid…

# Legacy code

In [5]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, widgets

# Ensure the renderer is set for the environment (especially helpful for Colab/VS Code)
import plotly.io as pio
pio.renderers.default = 'colab' # Change to 'notebook' if using classic Jupyter

def simulate_ai_future(harm_growth, benefit_ceiling, inst_speed, policy_lag):
    time = np.linspace(0, 25, 250)

    # 1. Exponential Harm Model
    y_harm = 0.5 * np.exp(harm_growth * time)

    # 2. Logistic Benefit Model (The 'Institutional' curve)
    # We add the policy_lag to the midpoint of the S-curve
    midpoint = 10 + policy_lag
    y_benefit = benefit_ceiling / (1 + np.exp(-inst_speed * (time - midpoint)))

    # 3. Calculate the "Velocity Gap" (Area between curves)
    # We only care about the area where Harm > Benefit
    gap_delta = np.maximum(0, y_harm - y_benefit)
    risk_score = np.trapz(gap_delta, time) # Numerical integration

    # 4. Create the Figure
    fig = go.Figure()

    # Harm Trace
    fig.add_trace(go.Scatter(
        x=time, y=y_harm, name='Harmful AI Potential',
        line=dict(color='#ef4444', width=3)
    ))

    # Benefit Trace
    fig.add_trace(go.Scatter(
        x=time, y=y_benefit, name='Realized AI Benefits',
        line=dict(color='#3b82f6', width=3),
        fill='tonexty', fillcolor='rgba(239, 68, 68, 0.1)'
    ))

    # Formatting
    fig.update_layout(
        title=f"<b>AI Velocity Gap Simulator</b><br>Cumulative Risk Score: {risk_score:.2f}",
        xaxis_title="Years from AGI Emergence",
        yaxis_title="Impact Magnitude",
        template="plotly_white",
        yaxis=dict(range=[0, min(max(y_harm)*1.1, 500)]), # Dynamic scaling
        annotations=[
            dict(x=20, y=max(y_harm)*0.8, text=f"Risk Area: {risk_score:.0f} units", showarrow=False)
        ]
    )

    fig.show()

# Create the interactive sliders
interact(
    simulate_ai_future,
    harm_growth = widgets.FloatSlider(value=0.25, min=0.1, max=0.4, step=0.01, description='Harm Growth:'),
    benefit_ceiling = widgets.IntSlider(value=50, min=10, max=100, step=5, description='Inst. Capacity:'),
    inst_speed = widgets.FloatSlider(value=0.4, min=0.1, max=1.0, step=0.05, description='Consensus Speed:'),
    policy_lag = widgets.IntSlider(value=0, min=-5, max=10, step=1, description='Policy Lag:')
);

interactive(children=(FloatSlider(value=0.25, description='Harm Growth:', max=0.4, min=0.1, step=0.01), IntSli…

In [6]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, widgets, Layout

# 1. Setup the basic plot structure
fig = go.FigureWidget()

# Add traces for Harm, Benefit, and the Gap area
fig.add_trace(go.Scatter(x=[], y=[], name='Harmful AI Potential',
                         line=dict(color='#ef4444', width=3)))
fig.add_trace(go.Scatter(x=[], y=[], name='Realized AI Benefits',
                         line=dict(color='#3b82f6', width=3),
                         fill='tonexty', fillcolor='rgba(239, 68, 68, 0.1)'))

fig.update_layout(
    title="<b>Interactive Velocity Gap Simulator</b><br>Exploring Institutional Friction vs. AI Growth",
    xaxis_title="Time (Years from AGI Emergence)",
    yaxis_title="Impact / Power",
    template="plotly_white",
    hovermode="x unified",
    yaxis=dict(range=[0, 100]),
    legend=dict(orientation="h", y=1.1)
)

# 2. Define the update function
def update_plot(harm_growth, benefit_ceiling, inst_speed, policy_lag):
    time = np.linspace(0, 25, 250)

    # Exponential Harm Model
    # y = e^(growth * time)
    y_harm = 0.5 * np.exp(harm_growth * time)

    # Logistic Benefit Model (Institutional constraints)
    # L / (1 + exp(-k * (t - x0)))
    y_benefit = benefit_ceiling / (1 + np.exp(-inst_speed * (time - (10 + policy_lag))))

    # Update data in the existing figure
    with fig.batch_update():
        fig.data[0].x = time
        fig.data[0].y = y_harm
        fig.data[1].x = time
        fig.data[1].y = y_benefit

        # Adjust Y-axis if growth explodes
        max_val = max(max(y_harm), benefit_ceiling)
        fig.update_layout(yaxis=dict(range=[0, min(max_val * 1.1, 500)]))

# 3. Create Sliders
style = {'description_width': 'initial'}
slider_layout = Layout(width='500px')

harm_slider = widgets.FloatSlider(value=0.25, min=0.1, max=0.45, step=0.01,
                                  description='AI Harm Growth Rate:', style=style, layout=slider_layout)
ceiling_slider = widgets.IntSlider(value=40, min=10, max=100, step=5,
                                   description='Institutional Capacity (Ceiling):', style=style, layout=slider_layout)
speed_slider = widgets.FloatSlider(value=0.4, min=0.1, max=1.0, step=0.05,
                                   description='Institutional Speed (Consensus):', style=style, layout=slider_layout)
lag_slider = widgets.IntSlider(value=0, min=-5, max=10, step=1,
                               description='Policy Delay (Lag):', style=style, layout=slider_layout)

# 4. Display the Interactive Dashboard
print("DIRECTIONS: Adjust sliders to see how Institutional Inertia creates the 'Velocity Gap'.")
ui = widgets.VBox([harm_slider, ceiling_slider, speed_slider, lag_slider])
out = widgets.interactive_output(update_plot, {
    'harm_growth': harm_slider,
    'benefit_ceiling': ceiling_slider,
    'inst_speed': speed_slider,
    'policy_lag': lag_slider
})

display(ui, fig)

DIRECTIONS: Adjust sliders to see how Institutional Inertia creates the 'Velocity Gap'.


FigureWidget({
    'data': [{'line': {'color': '#ef4444', 'width': 3},
              'name': 'Harmful AI Potential',
              'type': 'scatter',
              'uid': '87465396-e8f6-430e-9b6d-9d14078cde80',
              'x': array([ 0.        ,  0.10040161,  0.20080321, ..., 24.79919679, 24.89959839,
                          25.        ]),
              'y': array([  0.5       ,   0.51270903,   0.52574111, ..., 246.32505272,
                          252.5861599 , 259.00641233])},
             {'fill': 'tonexty',
              'fillcolor': 'rgba(239, 68, 68, 0.1)',
              'line': {'color': '#3b82f6', 'width': 3},
              'name': 'Realized AI Benefits',
              'type': 'scatter',
              'uid': '1e16ab9b-ca32-44ee-b6b5-e69de613698c',
              'x': array([ 0.        ,  0.10040161,  0.20080321, ..., 24.79919679, 24.89959839,
                          25.        ]),
              'y': array([ 0.7194484 ,  0.74837836,  0.77844858, ..., 39.8928453 , 39.897